# Set up

In [1]:
import torchreid
from torchreid.data.datasets.image.robo import Robo
torchreid.data.register_image_dataset('robo', Robo)

/notebooks/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='robo',
    transforms = "",
    use_gpu = False
)

Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
TRAIN:::  [('', 0, 0)]
QUERY:::  [('/notebooks/deep-person-reid/reid-data/sus/suspect.png', 0, 0), ('/notebooks/deep-person-reid/reid-data/sus/hostage.png', 0, 0)]
GALLERY:::  [('/notebooks/deep-person-reid/reid-data/test/0.png', 0, 2), ('/notebooks/deep-person-reid/reid-data/test/2.png', 0, 2), ('/notebooks/deep-person-reid/reid-data/test/1.png', 0, 2)]
=> Loaded Robo
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |     1 |        1 |         1
  query    |     1 |        2 |         1
  gallery  |     1 |        3 |         1
  ---------------------------

# Assemble Model

## model 1 - resnet50

In [ ]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

## mode 2 - resnet50mid

## model 4 - osnet_ain_x0_1

In [ ]:
model = torchreid.models.build_model(
    name="osnet_ain_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="triplet",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0015,
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

## model 5 - osnet_ain_x0_25

In [ ]:
model = torchreid.models.build_model(
    name="osnet_ain_x0_25",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

## model - osnet ibn x1_0

In [3]:
model = torchreid.models.build_model(
    name="osnet_ibn_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="triplet",
    pretrained=False # I DISABLED DOWNLOAD WEIGHTS!!!
)

# model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

# Assemble engine, load weights

In [4]:
engine = torchreid.engine.ImageTripletEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
    use_gpu=False
)

In [5]:
from torchreid.utils import load_pretrained_weights
weight_path = '/notebooks/deep-person-reid/log/til_osnet_ibn_x1_0_with_pretrained/model/model.pth.tar-10-95.7-460'
load_pretrained_weights(model, weight_path)

Successfully loaded pretrained weights from "/notebooks/deep-person-reid/log/til_osnet_ibn_x1_0_with_pretrained/model/model.pth.tar-10-95.7-460"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
# weight_path = '/notebooks/deep-person-reid/log/til_osnet_ibn_x1_0_with_pretrained/model/model.pth.tar-10-95.7-460'

In [ ]:
# load_pretrained_weights(model, weight_path)

In [6]:
_,distmat, queries_and_galley = engine.run(
    save_dir="log/til_resnet50",
    max_epoch=4,
    eval_freq=1,
    print_freq=50,
    test_only=True
)

##### Evaluating robo (source) #####
Extracting features from query set ...
HELLO HERE ARE UR PIDS AND CAMIDS: 
[0 0] [0 0]
Done, obtained 2-by-512 matrix
Extracting features from gallery set ...
HELLO HERE ARE UR PIDS AND CAMIDS: 
[0 0 0] [2 2 2]
Done, obtained 3-by-512 matrix
Speed: 1.8293 sec/batch
Computing distance matrix with metric=euclidean ...
YOUR DISTMAT!!!:  (2, 3)
[544.4458  611.84644 536.5933 ]
[628.04236 354.37045 730.9147 ]
Computing CMC and mAP ...
Note: number of gallery samples is quite small, got 3


In [ ]:
distmat

# post process results

In [ ]:
from PIL import Image

In [ ]:
def visualise_matches(d, q, index):
    query = q[0][index][0]
    distmat = d[index]
    gallery = q[1]
    
    temp = zip(distmat, gallery)
    print("#SUS ", query)
    im=Image.open(query)
    im.show()
    for i in sorted(temp):
        if query[-8:-4] in i[1][0][-13:-8]:
            print("#CANDIDATES: ", i)
            im=Image.open(i[1][0])
            im.show()

In [ ]:
def visualise_matches(d, q, index):
    query = q[0][index][0]
    distmat = d[index]
    gallery = q[1]
    
    temp = zip(distmat, gallery)
    print("#SUS ", query)
    im=Image.open(query)
    im.show()
    for i in sorted(temp):
        print("#CANDIDATES: ", i)
        im=Image.open(i[1][0])
        im.show()

for i in range(2):
    visualise_matches(distmat, queries_and_galley, i)

In [ ]:
def visualise_sorts(d, q, index):
    query = q[0][index][0]
    distmat = d[index]
    gallery = q[1]
    
    temp = zip(distmat, gallery)
    print("#SUS ", query)
    im=Image.open(query)
    im.show()
    count = 0
    for i in sorted(temp):
        if count <= 50:
            print("#SORTS: ", i)
            im=Image.open(i[1][0])
            im.show()
        
        if query[-8:-4] in i[1][0][-13:-8]:
            print("#CANDIDATES: ", i)
            im=Image.open(i[1][0])
            im.show()
        
        count += 1

In [ ]:
visualise_sorts(distmat, queries_and_galley, 10)

In [ ]:
def output_matches(d, q, index):
    query = q[0][index][0]
    distmat = d[index]
    gallery = q[1]
    
    temp = zip(distmat, gallery)
    # print(query)
    # im=Image.open(query)
    # im.show()
    
    output = {}
    for i in sorted(temp):
        if query[-8:-4] in i[1][0][-13:-8]:
            if i[0] < 455 and output == {}:
                output[int(i[1][0][-8:-4])] = 1
            else:
                output[int(i[1][0][-8:-4])] = 0
                
    return output

In [ ]:
import pandas as pd
from pandas import DataFrame
path = "/notebooks/deep-person-reid/reid-data/results (5).csv"
df = pd.read_csv(path)
df.head(10)

In [ ]:
for i in range(1600):
    output = output_matches(distmat, queries_and_galley, i)
    for j in output.keys():
        df.at[j,"class"] = output[int(j)]

In [ ]:
df.head(40)

In [ ]:
df.to_csv("/notebooks/file_transfer/submission_27.csv")

# Robotics challenge function

def targets_from_image(self, scene_img, target_img) -> BoundingBox:
        '''Process image with re-id pipeline and return the detected objects and their classes.
        
        Parameters
        ----------
        img : Any
            Input image.
        
        Returns
        -------
        results  : List[DetectedObject]
            List of DetectedObjects.
        '''
        
        # dummy data
        bbox = BoundingBox(100,100,300,50)
        obj = DetectedObject("1", "1", bbox)
        # DetectedObject: ['id', 'cls', 'bbox'] 
        # e.g. cls=0 means match class 0, cls=1 means match class 1, cls=2 means match class2.
        pass
        

In [ ]:
import pandas as pd
from PIL import Image
import os
from pathlib import Path
import shutil
from torchreid.utils import load_pretrained_weights

def crop_image(image_path, results_csv, candidate_save_to):
    df = pd.read_csv(results_csv)
    
    # Open the image using Pillow
    image = Image.open(image_path)
    w = image.width
    h = image.height
    
    # Ready to save, check directoreis
    if os.path.exists(candidate_save_to):
        shutil.rmtree(candidate_save_to)
    os.mkdir(candidate_save_to)
    
    # crop candidates
    for row in df.index:
        
        # obtain coordinates for one target
        ymin = df['ymin'][row]
        xmin = df['xmin'][row]
        ymax = df['ymax'][row]
        xmax = df['xmax'][row]
    
    
        # Compute x, y, width, height from params
        xmin = w * float(xmin)
        ymin = h * float(ymin) 
        xmax = w * float(xmax)
        ymax = h * float(ymax)

        # Crop the image using the provided coordinates
        cropped_image = image.crop((xmin, ymin, xmax, ymax))

        # Save the cropped image
        cropped_image.save(candidate_save_to + f"/{row}.png")
        

In [ ]:
import pandas as pd
df = pd.read_csv("/notebooks/deep-person-reid/reid-data/testcsv.csv")
df.head(5)

In [ ]:
image_path = "/notebooks/deep-person-reid/reid-data/image_0000.png"
results_csv = "/notebooks/deep-person-reid/reid-data/testcsv.csv"
save_to = "/notebooks/deep-person-reid/reid-data/test"
crop_image(image_path, results_csv, save_to)

In [ ]:
for i in os.listdir(save_to):
    image = Image.open(save_to + f"/{i}")
    image.show()

In [ ]:
def reid(scene_img, results_csv, candidate_save_to, engine): 
    df = pd.read_csv(results_csv)
    
    # process/crop images in the scene
    crop_image(image_path, results_csv, candidate_save_to)
    
    # target image should be stored in target path alrd
    
    # call test function, obtain matrix
    _,distmat, queries_and_galley = engine.run(
        save_dir="",
        max_epoch=4,
        eval_freq=1,
        print_freq=50,
        test_only=True
    )
    
    # for i in range(1600):
    #     output = output_matches(distmat, queries_and_galley, i)
    #     for j in output.keys():
    #         df.at[j,"class"] = output[int(j)]
    return distmat
    

In [ ]:
image_path = "/notebooks/deep-person-reid/reid-data/image_0000.png"
results_csv = "/notebooks/deep-person-reid/reid-data/testcsv.csv"
save_to = "/notebooks/deep-person-reid/reid-data/test"
reid(image_path, results_csv, save_to, engine)

In [ ]:
image_path = "/notebooks/deep-person-reid/reid-data/image_0000.png"

In [ ]:
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_ibn_x1_0',
    model_path='/notebooks/deep-person-reid/log/til_osnet_ibn_x1_0_with_pretrained/model/model.pth.tar-10-95.7-460',
    device='cpu'
)

In [ ]:
features = extractor(image_path)

In [ ]:
engine.model(features)